In [110]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=60

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    #c=rand(Float64,length(v))
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)
#f=f^2


# unit sphere constraint
m=1
q=floor(Int64,n/m)
R=ones(Float64,m)./m
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=60
Number of inequality constraints: m=1
Number of equality constraints: l=15


In [106]:
k=2
println("Relaxed order: k=",k)

Relaxed order: k=2


In [101]:
using ctpPOP

opt_val0=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 46376           
  Cones                  : 0               
  Scalar variables       : 3969            
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.12    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-4.622152744266953

In [111]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [112]:
using ctpPOP

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)
println("gap = ",abs(opt_val1-opt_val0)/abs(opt_val0)*100)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=2
  Size of the largest block: s^max=1891
  Number of equality trace constraints: zeta=1183767
Modeling time:
459.109499 seconds (1.26 G allocations: 652.542 GiB, 13.99% gc time)
iter=1   val=-12.968652924659343   gap=0.05221453901787956   feas=2.485395731772089
iter=2   val=-5.8208100194172205   gap=0.6588273517903134   feas=0.5931030035612227
iter=4   val=-6.543171334220503   gap=0.5048889589185458   feas=0.622906974145087
iter=8   val=-6.613949512786576   gap=0.402220446249998   feas=0.5145049166789029
iter=16   val=-6.177559895855624   gap=0.30392495924256496   feas=0.3774645081459468
iter=32   val=-5.372050326708486   gap=0.20674300359727457   feas=0.21109423497754864
iter=64   val=-4.9739087919509775   gap=0.04981380045538175   feas=0.09135426787423714
iter=128   val=-5.135727333147329   gap=0.0420802666923776   feas=0.02387544538242933
iter=256   val=-5.303497615416846   gap=0.0106377

In [113]:
using ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3,solver="COSMO")

println("gap = ",abs(opt_val2-opt_val0)/abs(opt_val0)*100)

------------------------------------------------------------------
          COSMO v0.7.8 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2020
------------------------------------------------------------------

Problem:  x ∈ R^{1819143},
          constraints: A ∈ R^{2426153x1819143} (57333230 nnz),
          matrix size to factor: 4245296x4245296,
          Floating-point precision: Float64
Sets:     PsdConeTriangle of dim: 1788886
          ZeroSet of dim: 635376
          PsdConeTriangle of dim: 1891
Settings: ϵ_abs = 1.0e-04, ϵ_rel = 1.0e-04,
          ϵ_prim_inf = 1.0e-06, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 2500,
          scaling iter = 10 (on),
          check termination every 40 iter,
          check infeasibility every 40 iter,
          KKT system solver: COSMO.QdldlKKTSolver
Setup Time: 4.88764642e6ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-1.780

In [16]:
# using ctpPOP

# opt_val=ctpPOP.POP_dense_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
#                          tol=1e-3,use_eqcons_to_get_constant_trace=false)

In [24]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)